In [1]:
import backtrader as bt
from datetime import datetime as dt
import pandas as pd
import yfinance
from backtrader.indicators import BollingerBands
import datetime 



In [2]:
import yfinance

sp500_data = yfinance.download('^GSPC', start='2000-01-01', end='2024-01-01')
sp500_data.to_csv('SP500_data_test.csv')

[*********************100%%**********************]  1 of 1 completed


In [3]:
class BB(bt.Strategy):
    params = (("period", 20), ("stddev", 2))
    
    def log(self, txt, dt=None):
        '''Logging Function'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.bollinger = BollingerBands(self.datas[0], period=self.params.period, devfactor=self.params.stddev)
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.buyprice = order.executed.price
                self.buycomm = order.executed.value
                self.log("BUY EXECUTED, Price: {}".format(order.executed.price))
                
            if order.issell():
                self.log("SELL EXECUTED, Price: {}".format(order.executed.price))
                
        self.order = None
        
    def next(self):
        self.log('Close, %.2f' % self.dataclose[0])
        
        if self.dataclose[0] > self.bollinger.lines.top[0] and not self.position:
            self.buy()
        elif self.dataclose[0] < self.bollinger.lines.bot[0] and self.position:
            self.sell()

In [4]:
class MeanReversionStrategy(bt.Strategy):
    params = (
        ('period', 20),          
        ('devfactor', 2),        
    )

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.bar_executed = 0
        
        self.sma = bt.indicators.SimpleMovingAverage(self.dataclose, period=self.params.period)
        self.stdev = bt.indicators.StandardDeviation(self.dataclose, period=self.params.period)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            elif order.issell():
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def next(self):
        if self.order:
            return

        mean = self.sma[0]
        std_dev = self.stdev[0]
        price = self.dataclose[0]

        self.log(f'Close: {price:.2f}, Mean: {mean:.2f}, StdDev: {std_dev:.2f}')

        if price < (mean - self.params.devfactor * std_dev):
            if not self.position:  # Only buy if not already in a position
                self.order = self.buy()
                self.log(f'BUY CREATE, Price: {price:.2f}')
        
        elif price > (mean + self.params.devfactor * std_dev):
            if self.position:  # Only sell if currently in a position
                self.order = self.sell()
                self.log(f'SELL CREATE, Price: {price:.2f}')

In [5]:
class MACD(bt.Strategy):
    params = (
        ('fast_ema_period', 12),
        ('slow_ema_period', 26),
        ('signal_period', 9),
    )
    
    def log(self, txt, dt=None):
        '''Logging function'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.bar_executed = 0
        
        # Add MACD indicator
        self.macd = bt.indicators.MACD(
            self.datas[0],
            period_me1=self.params.fast_ema_period,
            period_me2=self.params.slow_ema_period,
            period_signal=self.params.signal_period
        )
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        self.order = None
        
    def next(self):
        self.log('Close, %.2f' % self.dataclose[0])
        
        if self.order:
            return
        
        if not self.position:
            if self.macd.macd[0] > self.macd.signal[0]:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
        else:
            if self.macd.macd[0] < self.macd.signal[0]:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()

In [6]:
class MovingAverageCrossover(bt.Strategy):
    
    params = (
        ('short_period', 50),
        ('long_period', 200),  
    )
    
    def log(self, txt, dt=None):
        '''Logging Function'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.short_sma = bt.indicators.SMA(self.data, period=self.p.short_period)
        self.long_sma = bt.indicators.SMA(self.data, period=self.p.long_period)
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        
        if order.status == order.Completed:
            if order.isbuy():
                self.log("BUY EXECUTED, Price: {}".format(order.executed.price))
            elif order.issell():
                self.log("SELL EXECUTED, Price: {}".format(order.executed.price))
        
        self.order = None
        
    def next(self):
        self.log('Close, %.2f' % self.dataclose[0])
        
        #Prevents buying/selling stock if there is an order being made
        if self.order:
            return
        
        
        if self.short_sma > self.long_sma and not self.position:
            self.log('BUY CREATED, %.2f' % self.dataclose[0])
            self.order = self.buy()
        elif self.short_sma < self.long_sma and self.position:
            self.log("SELL CREATED, %.2f" % self.dataclose[0])
            self.order = self.sell()

In [7]:
class RSI(bt.Strategy):
    params = (('period', 14),)
    
    def log(self, txt, dt=None):
        '''Logging Function'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.rsi = bt.indicators.RelativeStrengthIndex(self.data, period=self.params.period)
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        
        if order.status == order.Completed:
            if order.isbuy():
                self.log("BUY EXECUTED, Price: {}".format(order.executed.price))
            elif order.issell():
                self.log("SELL EXECUTED, Price: {}".format(order.executed.price))
        
        self.order = None
        
    def next(self):
        self.log('Close, %.2f' % self.dataclose[0])
        
        #Prevents buying/selling stock if there is an order being made
        if self.order:
            return
        
        if self.rsi < 30 and not self.position:
            self.log('BUY CREATED, %.2f' % self.dataclose[0])
            self.order = self.buy()
        elif self.rsi > 70 and self.position:
            self.log("SELL CREATED, %.2f" % self.dataclose[0])
            self.order = self.sell()

In [8]:
cerebro = bt.Cerebro()

cerebro.broker.set_cash(1000000)

data = bt.feeds.YahooFinanceCSVData(
    dataname= r"SP500_data_test.csv",
    fromdate = datetime.datetime(2000,1,1),
    todate = datetime.datetime(2024,1,1),
    reverse=False)

cerebro.adddata(data)
cerebro.addstrategy(RSI)
cerebro.addsizer(bt.sizers.FixedSize, stake=500)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.plot()

Starting Portfolio Value: 1000000.00
2000-01-24, Close, 1401.53
2000-01-25, Close, 1410.03
2000-01-26, Close, 1404.09
2000-01-27, Close, 1398.56
2000-01-28, Close, 1360.16
2000-01-31, Close, 1394.46
2000-02-01, Close, 1409.28
2000-02-02, Close, 1409.12
2000-02-03, Close, 1424.97
2000-02-04, Close, 1424.37
2000-02-07, Close, 1424.24
2000-02-08, Close, 1441.72
2000-02-09, Close, 1411.71
2000-02-10, Close, 1416.83
2000-02-11, Close, 1387.12
2000-02-14, Close, 1389.94
2000-02-15, Close, 1402.05
2000-02-16, Close, 1387.67
2000-02-17, Close, 1388.26
2000-02-18, Close, 1346.09
2000-02-22, Close, 1352.17
2000-02-23, Close, 1360.69
2000-02-24, Close, 1353.43
2000-02-25, Close, 1333.36
2000-02-28, Close, 1348.05
2000-02-29, Close, 1366.42
2000-03-01, Close, 1379.19
2000-03-02, Close, 1381.76
2000-03-03, Close, 1409.17
2000-03-06, Close, 1391.28
2000-03-07, Close, 1355.62
2000-03-08, Close, 1366.70
2000-03-09, Close, 1401.69
2000-03-10, Close, 1395.07
2000-03-13, Close, 1383.62
2000-03-14, Close,

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 5 Axes>]]